# Language Translation
In this project, we going to take a peek into the realm of neural network machine translation. we’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we will be taking a small portion of english corpus.

## Following are some helper functions

In [1]:
import os
import pickle
import copy
import numpy as np

In [2]:

CODES = {'<PAD>': 0, '<EOS>': 1, '<UNK>': 2, '<GO>': 3 }


def load_data(path):
    """
    Load Dataset from File
    """
    input_file = os.path.join(path)
    with open(input_file, 'r', encoding='utf-8') as f:
        data = f.read()

    return data


def preprocess_and_save_data(source_path, target_path, text_to_ids):
    """
    Preprocess Text Data.  Save to to file.
    """
    # Preprocess
    source_text = load_data(source_path)
    target_text = load_data(target_path)

    source_text = source_text.lower()
    target_text = target_text.lower()

    source_vocab_to_int, source_int_to_vocab = create_lookup_tables(source_text)
    target_vocab_to_int, target_int_to_vocab = create_lookup_tables(target_text)

    source_text, target_text = text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int)

    # Save Data
    pickle.dump((
        (source_text, target_text),
        (source_vocab_to_int, target_vocab_to_int),
        (source_int_to_vocab, target_int_to_vocab)), open('preprocess.p', 'wb'))


def load_preprocess():
    """
    Load the Preprocessed Training data and return them in batches of <batch_size> or less
    """
    return pickle.load(open('preprocess.p', mode='rb'))


def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    """
    vocab = set(text.split())
    vocab_to_int = copy.copy(CODES)

    for v_i, v in enumerate(vocab, len(CODES)):
        vocab_to_int[v] = v_i

    int_to_vocab = {v_i: v for v, v_i in vocab_to_int.items()}

    return vocab_to_int, int_to_vocab


def save_params(params):
    """
    Save parameters to file
    """
    pickle.dump(params, open('params.p', 'wb'))


def load_params():
    """
    Load parameters from file
    """
    return pickle.load(open('params.p', mode='rb'))


def batch_data(source, target, batch_size):
    """
    Batch source and target together
    """
    for batch_i in range(0, len(source)//batch_size):
        start_i = batch_i * batch_size
        source_batch = source[start_i:start_i + batch_size]
        target_batch = target[start_i:start_i + batch_size]
        yield np.array(pad_sentence_batch(source_batch)), np.array(pad_sentence_batch(target_batch))


def pad_sentence_batch(sentence_batch):
    """
    Pad sentence with <PAD> id
    """
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    print(max_sentence)
    return [sentence + [CODES['<PAD>']] * (max_sentence - len(sentence))
            for sentence in sentence_batch]


In [3]:
# Load data
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = load_data(source_path)
target_text = load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [4]:
view_sentence_range = (0, 10)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))


sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

# Solution 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of each sentence from `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [5]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # Just go through the text and transform it.
    source_id_text = []
    for idx, line in enumerate(source_text.split('\n')):
        source_id_text.append([])
        for word in line.split():
            source_id_text[idx].append(source_vocab_to_int[word])
        
    target_id_text = []
    for idx, line in enumerate(target_text.split('\n')):
        target_id_text.append([])
        for word in line.split():
            target_id_text[idx].append(target_vocab_to_int[word])
        target_id_text[idx].append(target_vocab_to_int['<EOS>'])

    return (source_id_text, target_id_text)



### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
preprocess_and_save_data(source_path, target_path, text_to_ids)

## Check Point
This is the first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = load_preprocess()

## Build the Neural Network
we'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoding_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.

Return the placeholders in the following the tuple (Input, Targets, Learing Rate, Keep Probability)

In [8]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, shape=[None,None], name= "input")
    targets = tf.placeholder(tf.int32, shape=[None,None], name= "targets")
    lrate = tf.placeholder(tf.float32, name= "learning_rate")
    keep_prob = tf.placeholder(tf.float32, name= "keep_prob")
    target_seq_lenth = tf.placeholder(tf.int32, shape=[None], name= "target_sequence_length")
    max_target_len = tf.reduce_max(target_seq_lenth, name= 'max_target_len')
    source_seq_length = tf.placeholder(tf.int32, shape=[None], name= "source_sequence_length")
    return (inputs, targets, lrate, keep_prob, target_seq_lenth, max_target_len, source_seq_length)


### Process Decoder Input
Implement `process_decoding_input` using TensorFlow to remove the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [9]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # Create a constant tensor with the 'go id'.
    go_id = tf.constant(target_vocab_to_int['<GO>'], shape=(batch_size,1), dtype=tf.int32)
    # Concatenate the vector without the last word id with the go ids vector
    processed_input = tf.concat([go_id,target_data[:,:-1]],1)
    return processed_input

### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer using [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn).

In [10]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # Build the lstm cells wrapped in dropout
    def build_cell(rnn_size, keep_prob):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        lstm_drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return lstm_drop
    # Stack them all
    stacked_lstm = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    # Creathe embedding layer.
    embed_encoder = tf.contrib.layers.embed_sequence(rnn_inputs, vocab_size = source_vocab_size, embed_dim = encoding_embedding_size)
    # If we don't have an initial zero state, provide a dtype.
    output, state = tf.nn.dynamic_rnn(stacked_lstm, embed_encoder, source_sequence_length, dtype=tf.float32)
    return (output, state)


### Decoding - Training
Create training logits using [`tf.contrib.seq2seq.simple_decoder_fn_train()`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/simple_decoder_fn_train) and [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder).  Apply the `output_fn` to the [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder) outputs.

In [11]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    trainig_helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, target_sequence_length)
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, trainig_helper, encoder_state, output_layer)
    f_output, _, _ = tf.contrib.seq2seq.dynamic_decode(basic_decoder,maximum_iterations=max_summary_length)
    return f_output


### Decoding - Inference
Create inference logits using [`tf.contrib.seq2seq.simple_decoder_fn_inference()`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/simple_decoder_fn_inference) and [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder). 

In [12]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # Convert the start_ids to be a vector with batch size (the go id repeated batch size times)
    start_ids = tf.tile([start_of_sequence_id], [batch_size])
    # Create the embedding helper.
    embedding_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
        dec_embeddings, start_ids, end_of_sequence_id)
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(
        dec_cell, embedding_helper, encoder_state, output_layer)
    f_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
        basic_decoder,maximum_iterations=max_target_sequence_length)
    return f_output


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

- Create RNN cell for decoding using `rnn_size` and `num_layers`.
- Create the output fuction using [`lambda`](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions) to transform it's input, logits, to class logits.
- Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope, output_fn, keep_prob)` function to get the training logits.
- Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, maximum_length, vocab_size, decoding_scope, output_fn, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [13]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # Use the same proess as in the encoding layer.
    def build_cell(rnn_size, keep_prob):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        lstm_drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return lstm_drop
    # Stack them all
    stacked_lstm = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    dense_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    with tf.variable_scope("decode") as scope:
        tr_decoder_output = decoding_layer_train(
            encoder_state, stacked_lstm, dec_embed_input, 
            target_sequence_length, max_target_sequence_length, 
            dense_layer, keep_prob)
        scope.reuse_variables()
        inf_decoder_output = decoding_layer_infer(
            encoder_state, stacked_lstm, dec_embeddings, 
            target_vocab_to_int['<GO>'], target_vocab_to_int['<EOS>'], 
            max_target_sequence_length, target_vocab_size, 
            dense_layer, batch_size, keep_prob)
    
    return tr_decoder_output, inf_decoder_output


### Build the Neural Network
Apply the functions you implemented above to:

- Apply embedding to the input data for the encoder.
- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob)`.
- Process target data using your `process_decoding_input(target_data, target_vocab_to_int, batch_size)` function.
- Apply embedding to the target data for the decoder.
- Decode the encoded input using your `decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size, num_layers, target_vocab_to_int, keep_prob)`.

In [14]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    output, state = encoding_layer(input_data, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   enc_embedding_size)
    
    processed_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    tr_decoder_output, inf_decoder_output = decoding_layer(processed_input, state,
                   target_sequence_length, max_target_sentence_length,
                   rnn_size, num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, dec_embedding_size)
    
    return tr_decoder_output, inf_decoder_output

## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability

In [15]:
# Number of Epochs
epochs = 10
# Batch Size
batch_size = 512
# RNN Size
rnn_size = 128
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 128
decoding_embedding_size = 128
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.55
display_step = True

### Build the Graph
Build the graph using the neural network you implemented.

In [16]:
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths

### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [19]:

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    1/269 - Train Accuracy: 0.2329, Validation Accuracy: 0.3096, Loss: 5.6859
Epoch   0 Batch    2/269 - Train Accuracy: 0.2655, Validation Accuracy: 0.3096, Loss: 5.4589
Epoch   0 Batch    3/269 - Train Accuracy: 0.2444, Validation Accuracy: 0.3096, Loss: 5.3048
Epoch   0 Batch    4/269 - Train Accuracy: 0.2317, Validation Accuracy: 0.3096, Loss: 5.1488
Epoch   0 Batch    5/269 - Train Accuracy: 0.2325, Validation Accuracy: 0.3097, Loss: 4.9733
Epoch   0 Batch    6/269 - Train Accuracy: 0.2867, Validation Accuracy: 0.3184, Loss: 4.6426
Epoch   0 Batch    7/269 - Train Accuracy: 0.2877, Validation Accuracy: 0.3220, Loss: 4.5144
Epoch   0 Batch    8/269 - Train Accuracy: 0.2558, Validation Accuracy: 0.3237, Loss: 4.5537
Epoch   0 Batch    9/269 - Train Accuracy: 0.2984, Validation Accuracy: 0.3399, Loss: 4.3468
Epoch   0 Batch   10/269 - Train Accuracy: 0.2680, Validation Accuracy: 0.3411, Loss: 4.3995
Epoch   0 Batch   11/269 - Train Accuracy: 0.3065, Validation Accuracy

Epoch   0 Batch   90/269 - Train Accuracy: 0.4322, Validation Accuracy: 0.4899, Loss: 2.6045
Epoch   0 Batch   91/269 - Train Accuracy: 0.4550, Validation Accuracy: 0.4846, Loss: 2.4578
Epoch   0 Batch   92/269 - Train Accuracy: 0.4658, Validation Accuracy: 0.4925, Loss: 2.4381
Epoch   0 Batch   93/269 - Train Accuracy: 0.4869, Validation Accuracy: 0.4893, Loss: 2.3445
Epoch   0 Batch   94/269 - Train Accuracy: 0.4547, Validation Accuracy: 0.4827, Loss: 2.4198
Epoch   0 Batch   95/269 - Train Accuracy: 0.4736, Validation Accuracy: 0.4896, Loss: 2.4187
Epoch   0 Batch   96/269 - Train Accuracy: 0.4648, Validation Accuracy: 0.4889, Loss: 2.4030
Epoch   0 Batch   97/269 - Train Accuracy: 0.4642, Validation Accuracy: 0.4901, Loss: 2.3944
Epoch   0 Batch   98/269 - Train Accuracy: 0.4855, Validation Accuracy: 0.4894, Loss: 2.3608
Epoch   0 Batch   99/269 - Train Accuracy: 0.4370, Validation Accuracy: 0.4875, Loss: 2.5079
Epoch   0 Batch  100/269 - Train Accuracy: 0.4876, Validation Accuracy

Epoch   0 Batch  179/269 - Train Accuracy: 0.4825, Validation Accuracy: 0.5025, Loss: 1.7978
Epoch   0 Batch  180/269 - Train Accuracy: 0.4772, Validation Accuracy: 0.5022, Loss: 1.7781
Epoch   0 Batch  181/269 - Train Accuracy: 0.4696, Validation Accuracy: 0.4966, Loss: 1.7763
Epoch   0 Batch  182/269 - Train Accuracy: 0.4776, Validation Accuracy: 0.5045, Loss: 1.7828
Epoch   0 Batch  183/269 - Train Accuracy: 0.5478, Validation Accuracy: 0.5063, Loss: 1.5442
Epoch   0 Batch  184/269 - Train Accuracy: 0.4408, Validation Accuracy: 0.4923, Loss: 1.8302
Epoch   0 Batch  185/269 - Train Accuracy: 0.5016, Validation Accuracy: 0.5122, Loss: 1.7545
Epoch   0 Batch  186/269 - Train Accuracy: 0.4562, Validation Accuracy: 0.5093, Loss: 1.8217
Epoch   0 Batch  187/269 - Train Accuracy: 0.4857, Validation Accuracy: 0.4982, Loss: 1.7026
Epoch   0 Batch  188/269 - Train Accuracy: 0.5007, Validation Accuracy: 0.5075, Loss: 1.6914
Epoch   0 Batch  189/269 - Train Accuracy: 0.4932, Validation Accuracy

Epoch   1 Batch    1/269 - Train Accuracy: 0.4810, Validation Accuracy: 0.5305, Loss: 1.4724
Epoch   1 Batch    2/269 - Train Accuracy: 0.4864, Validation Accuracy: 0.5273, Loss: 1.4426
Epoch   1 Batch    3/269 - Train Accuracy: 0.4788, Validation Accuracy: 0.5231, Loss: 1.4698
Epoch   1 Batch    4/269 - Train Accuracy: 0.4829, Validation Accuracy: 0.5305, Loss: 1.4539
Epoch   1 Batch    5/269 - Train Accuracy: 0.4730, Validation Accuracy: 0.5294, Loss: 1.4723
Epoch   1 Batch    6/269 - Train Accuracy: 0.5099, Validation Accuracy: 0.5273, Loss: 1.3476
Epoch   1 Batch    7/269 - Train Accuracy: 0.5156, Validation Accuracy: 0.5344, Loss: 1.3889
Epoch   1 Batch    8/269 - Train Accuracy: 0.4892, Validation Accuracy: 0.5352, Loss: 1.4508
Epoch   1 Batch    9/269 - Train Accuracy: 0.4983, Validation Accuracy: 0.5336, Loss: 1.3987
Epoch   1 Batch   10/269 - Train Accuracy: 0.4748, Validation Accuracy: 0.5323, Loss: 1.4205
Epoch   1 Batch   11/269 - Train Accuracy: 0.5099, Validation Accuracy

Epoch   1 Batch   90/269 - Train Accuracy: 0.5029, Validation Accuracy: 0.5509, Loss: 1.2408
Epoch   1 Batch   91/269 - Train Accuracy: 0.5272, Validation Accuracy: 0.5506, Loss: 1.1536
Epoch   1 Batch   92/269 - Train Accuracy: 0.5298, Validation Accuracy: 0.5456, Loss: 1.1548
Epoch   1 Batch   93/269 - Train Accuracy: 0.5431, Validation Accuracy: 0.5498, Loss: 1.1148
Epoch   1 Batch   94/269 - Train Accuracy: 0.5310, Validation Accuracy: 0.5502, Loss: 1.1707
Epoch   1 Batch   95/269 - Train Accuracy: 0.5357, Validation Accuracy: 0.5496, Loss: 1.1493
Epoch   1 Batch   96/269 - Train Accuracy: 0.5359, Validation Accuracy: 0.5532, Loss: 1.1330
Epoch   1 Batch   97/269 - Train Accuracy: 0.5180, Validation Accuracy: 0.5520, Loss: 1.1437
Epoch   1 Batch   98/269 - Train Accuracy: 0.5585, Validation Accuracy: 0.5541, Loss: 1.1283
Epoch   1 Batch   99/269 - Train Accuracy: 0.5151, Validation Accuracy: 0.5546, Loss: 1.1894
Epoch   1 Batch  100/269 - Train Accuracy: 0.5534, Validation Accuracy

...

Epoch   8 Batch  179/269 - Train Accuracy: 0.8835, Validation Accuracy: 0.8911, Loss: 0.1879
Epoch   8 Batch  180/269 - Train Accuracy: 0.9020, Validation Accuracy: 0.8937, Loss: 0.1860
Epoch   8 Batch  181/269 - Train Accuracy: 0.8861, Validation Accuracy: 0.8843, Loss: 0.1966
Epoch   8 Batch  182/269 - Train Accuracy: 0.9000, Validation Accuracy: 0.8866, Loss: 0.1910
Epoch   8 Batch  183/269 - Train Accuracy: 0.9058, Validation Accuracy: 0.8805, Loss: 0.1589
Epoch   8 Batch  184/269 - Train Accuracy: 0.8839, Validation Accuracy: 0.8791, Loss: 0.1931
Epoch   8 Batch  185/269 - Train Accuracy: 0.8857, Validation Accuracy: 0.8780, Loss: 0.1911
Epoch   8 Batch  186/269 - Train Accuracy: 0.8760, Validation Accuracy: 0.8825, Loss: 0.1865
Epoch   8 Batch  187/269 - Train Accuracy: 0.8780, Validation Accuracy: 0.8848, Loss: 0.1932
Epoch   8 Batch  188/269 - Train Accuracy: 0.8956, Validation Accuracy: 0.8880, Loss: 0.1784
Epoch   8 Batch  189/269 - Train Accuracy: 0.8832, Validation Accuracy

Epoch   9 Batch    1/269 - Train Accuracy: 0.8967, Validation Accuracy: 0.8965, Loss: 0.1803
Epoch   9 Batch    2/269 - Train Accuracy: 0.8864, Validation Accuracy: 0.8917, Loss: 0.1776
Epoch   9 Batch    3/269 - Train Accuracy: 0.8987, Validation Accuracy: 0.8866, Loss: 0.1781
Epoch   9 Batch    4/269 - Train Accuracy: 0.8699, Validation Accuracy: 0.8806, Loss: 0.1810
Epoch   9 Batch    5/269 - Train Accuracy: 0.8974, Validation Accuracy: 0.8873, Loss: 0.1887
Epoch   9 Batch    6/269 - Train Accuracy: 0.9068, Validation Accuracy: 0.8865, Loss: 0.1692
Epoch   9 Batch    7/269 - Train Accuracy: 0.8795, Validation Accuracy: 0.8825, Loss: 0.1701
Epoch   9 Batch    8/269 - Train Accuracy: 0.9012, Validation Accuracy: 0.8908, Loss: 0.1809
Epoch   9 Batch    9/269 - Train Accuracy: 0.8806, Validation Accuracy: 0.8789, Loss: 0.1806
Epoch   9 Batch   10/269 - Train Accuracy: 0.8952, Validation Accuracy: 0.8875, Loss: 0.1727
Epoch   9 Batch   11/269 - Train Accuracy: 0.8959, Validation Accuracy

Epoch   9 Batch   90/269 - Train Accuracy: 0.9020, Validation Accuracy: 0.9029, Loss: 0.1630
Epoch   9 Batch   91/269 - Train Accuracy: 0.9188, Validation Accuracy: 0.8973, Loss: 0.1507
Epoch   9 Batch   92/269 - Train Accuracy: 0.9058, Validation Accuracy: 0.8968, Loss: 0.1564
Epoch   9 Batch   93/269 - Train Accuracy: 0.9139, Validation Accuracy: 0.8923, Loss: 0.1526
Epoch   9 Batch   94/269 - Train Accuracy: 0.8942, Validation Accuracy: 0.8903, Loss: 0.1672
Epoch   9 Batch   95/269 - Train Accuracy: 0.9005, Validation Accuracy: 0.8975, Loss: 0.1536
Epoch   9 Batch   96/269 - Train Accuracy: 0.8831, Validation Accuracy: 0.8993, Loss: 0.1601
Epoch   9 Batch   97/269 - Train Accuracy: 0.8915, Validation Accuracy: 0.8964, Loss: 0.1594
Epoch   9 Batch   98/269 - Train Accuracy: 0.8996, Validation Accuracy: 0.8994, Loss: 0.1567
Epoch   9 Batch   99/269 - Train Accuracy: 0.9015, Validation Accuracy: 0.9025, Loss: 0.1549
Epoch   9 Batch  100/269 - Train Accuracy: 0.9135, Validation Accuracy

Epoch   9 Batch  179/269 - Train Accuracy: 0.8948, Validation Accuracy: 0.9025, Loss: 0.1478
Epoch   9 Batch  180/269 - Train Accuracy: 0.9119, Validation Accuracy: 0.9069, Loss: 0.1413
Epoch   9 Batch  181/269 - Train Accuracy: 0.9047, Validation Accuracy: 0.9049, Loss: 0.1551
Epoch   9 Batch  182/269 - Train Accuracy: 0.9063, Validation Accuracy: 0.9065, Loss: 0.1476
Epoch   9 Batch  183/269 - Train Accuracy: 0.9179, Validation Accuracy: 0.9041, Loss: 0.1286
Epoch   9 Batch  184/269 - Train Accuracy: 0.9115, Validation Accuracy: 0.9062, Loss: 0.1487
Epoch   9 Batch  185/269 - Train Accuracy: 0.9146, Validation Accuracy: 0.9049, Loss: 0.1528
Epoch   9 Batch  186/269 - Train Accuracy: 0.9141, Validation Accuracy: 0.9111, Loss: 0.1432
Epoch   9 Batch  187/269 - Train Accuracy: 0.9063, Validation Accuracy: 0.9064, Loss: 0.1506
Epoch   9 Batch  188/269 - Train Accuracy: 0.9113, Validation Accuracy: 0.9090, Loss: 0.1338
Epoch   9 Batch  189/269 - Train Accuracy: 0.8930, Validation Accuracy

Model Trained and Saved


### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [21]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
save_params(save_path)

In [22]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [24]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    word_ids = []
    for word in sentence.lower().split():
        if word in vocab_to_int:
            word_ids.append(vocab_to_int[word])
        else:
            word_ids.append(vocab_to_int['<UNK>'])
    return word_ids

## Translate
This will translate `translate_sentence` from English to French.

In [25]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))

INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [120, 172, 10, 36, 151, 179, 186]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [182, 99, 65, 195, 282, 5, 243, 69, 1]
  French Words: il a de visiter le camion automne . <EOS>
